# Unit 3.1 - Classical Cryptography Problems  
_Number‑theory primer for Shor’s algorithm_

**What you will learn**

* How modular arithmetic forms finite (cyclic) groups.  
* Why fast integer multiplication (Schönhage-Strassen) relies on the discrete Fourier transform (DFT).  
* How period‑finding in a cyclic group underpins Shor’s quantum speed‑up.

> **Prerequisites:** basic Python, NumPy, complex numbers.

In [ ]:
# Cell 1 - Imports
import numpy as np
from math import gcd

## 1  Cyclic groups mod $N$

### 1.1 Additive group $\mathbb Z_N$

For a modulus $N$ the set $\{0,1,\dots,N-1\}$ with addition mod $N$ is a cyclic group.


In [ ]:
N = 7
add_group = [(a, b, (a + b) % N) for a in range(N) for b in range(N)]
add_group[:5]  # first five triples (a,b,a⊕b)

### 1.2 Multiplicative group $\mathbb Z_N^{\times}$

For prime $N$ the non‑zero elements form a cyclic multiplicative group of order $N-1$.  
We search for a generator $g$ such that $g^k \bmod N$ cycles through all elements.

In [ ]:
def multiplicative_group(N):
    return [a for a in range(1, N) if gcd(a, N) == 1]

def find_generator(N):
    group = multiplicative_group(N)
    for g in group:
        if len({pow(g, k, N) for k in range(1, N)}) == N - 1:
            return g
    return None

N = 7
g = find_generator(N)
print(f"Multiplicative group generators mod {N}: {g}")

## 2  Schönhage-Strassen: convolution = FFT

Fast integer multiplication reduces to polynomial convolution, which is accelerated
by the discrete Fourier transform (DFT).  
Below, two base-$B$ numbers are multiplied via convolution of their digit vectors.

In [ ]:
# Cell 2 - Naïve vs FFT-based multiplication
from numpy.fft import fft, ifft

def schoolbook(a, b, B=10):
    return int(a) * int(b)

def fft_multiply(a, b, B=10):
    # convert to coefficient list
    A = [int(d) for d in str(a)][::-1]
    Bv = [int(d) for d in str(b)][::-1]
    size = len(A) + len(Bv)
    fa = fft(A, size)
    fb = fft(Bv, size)
    coeffs = np.round(ifft(fa * fb).real).astype(int)
    # carry handling
    carry = 0
    for i, c in enumerate(coeffs):
        total = c + carry
        coeffs[i] = total % 10
        carry = total // 10
    while carry:
        coeffs = np.append(coeffs, carry % 10)
        carry //= 10
    return int("".join(map(str, coeffs[::-1])))

a, b = 123456, 987654
print("schoolbook :", schoolbook(a, b))
print("FFT result :", fft_multiply(a, b))

## 3  Period‑finding intuition

Given $f(x)=g^x \bmod N$, Shor’s algorithm relies on finding the smallest $r$
such that $g^r \equiv 1 \pmod N$.

### 3.1 Classical trial division

In [ ]:
def period(g, N):
    r = 1
    val = pow(g, r, N)
    while val != 1:
        r += 1
        val = pow(g, r, N)
    return r

N = 15
g = 7
print(f"Period of {g} mod {N} is {period(g, N)}")

### 3.2 Plot $f(x)=g^x \bmod N$ and observe periodicity

In [ ]:
import matplotlib.pyplot as plt
xs  = np.arange(20)
ys  = [pow(g, x, N) for x in xs]
plt.stem(xs, ys, use_line_collection=True)
plt.title(f"Values of $g^x \\bmod N$ (g={g}, N={N})")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.show()

## 4  Take‑aways

* Cyclic groups under addition or multiplication mod \(N$ set the stage for
  order‑finding.  
* Schönhage-Strassen shows how Fourier analysis accelerates classical arithmetic;
  its quantum analogue uses the quantum Fourier transform.  
* Period‑finding over $\mathbb Z_N^{\times}$ is the heart of Shor’s exponential
  speed‑up for factoring.

### Further exercises

1. Write a function to compute discrete logs by brute force and test how the
   running time scales with $\varphi(N)$.  
2. Replace the DFT with the quantum Fourier transform (QFT) on a simulator
   for $N=15$ using Qiskit.